# Électricité & GES — Pipeline (API dynamique, mapping EXACT)

Ordre: **(1)** normaliser adresses → **(2)** lier aux IDs (logique EXACTE) → **(3)** agrégations 5 ans (GES + consommations) → **(4)** export **unique** CSV.

## 0) Imports & setup

In [1]:

import sys
from pathlib import Path
parent = Path.cwd().parent
if (parent / "pipeline_utils.py").exists():
    sys.path.append(str(parent))

import pandas as pd
from pipeline_utils import (
    fetch_ckan_resource, combine_energy_apis,
    ensure_address_columns, make_ges_column, make_consumption_columns,
    aggregate_by_year, map_municipal_id,  build_5y_means, finalize_export_5y_means
    
)

YEARS = [2019, 2020, 2021, 2022, 2023]


## 1) API CKAN — URLs

In [2]:

# Énergie: 2019+2020, 2021, 2022, 2023
api_energy = [
    "https://donnees.montreal.ca/api/3/action/datastore_search?resource_id=93bb6d04-6b86-4ffd-951c-e8119933fcb5",
    "https://donnees.montreal.ca/api/3/action/datastore_search?resource_id=44a7750a-47a0-4c67-895c-ed2e765b0bc0",
    "https://donnees.montreal.ca/api/3/action/datastore_search?resource_id=42b5bba6-6bbe-4ccf-b459-5df7bb95db96",
    "https://donnees.montreal.ca/api/3/action/datastore_search?resource_id=56a7aafc-2839-4748-8c45-e87a61226c5e",
]
default_years = [None, 2021, 2022, 2023]

# Bâtiments municipaux
api_muni = "https://donnees.montreal.ca/api/3/action/datastore_search?resource_id=1dfeb734-2b5e-47a9-ab3b-fb55e67b99f0"


## 2) Téléchargement dynamique (sans cache)

In [3]:

energy_df = combine_energy_apis(api_energy, default_years)
batiments_df = fetch_ckan_resource(api_muni)
print("Énergie:", len(energy_df), "— Bâtiments:", len(batiments_df))


Énergie: 580 — Bâtiments: 2075


## 3) Normalisation colonnes & métriques

In [4]:

addr_col = "Adresse_civique"
year_col = "Annee_consommation"

energy_df = ensure_address_columns(energy_df, addr_col=addr_col)
# Année déjà harmonisée dans combine_energy_apis
energy_df = make_ges_column(energy_df)
energy_df = make_consumption_columns(energy_df)
energy_df[[addr_col, "Adresse_norm", year_col]].head()


,Adresse_civique,Adresse_norm,Annee_consommation
0,4101 Rue Sherbrooke E,4101 rue sherbrooke est,2019
1,2269 Rue Viau,2269 rue viau,2019
2,2980 Rue Viau,2980 rue viau,2019
3,775 Rue Gosford,775 rue gosford,2019
4,5485 Chemin de la Côte-Saint-Paul,5485 chemin de la cote-saint-paul,2019


## 4) Mapping EXACT des IDs municipaux (même logique)

In [5]:
# --- Extraction des adresses uniques pour le mapping ---
core_tmp = (
    energy_df[["Adresse_norm", "Adresse_civique"]]
    .dropna(subset=["Adresse_norm"])
    .drop_duplicates("Adresse_norm", keep="first")
)

# --- Mapping "legacy" : reproduit la logique de l'ancien notebook (exact -> exact_norm -> fuzzy_strict -> fuzzy_loose)
core_with_id = map_municipal_id(
    core_tmp,
    batiments_df,
    enable_fuzzy_strict=True,   # active la correspondance token_set ≥90
    enable_fuzzy_loose=True     # active la correspondance token_set ≥85
)

# --- Vérification des correspondances obtenues ---
print(core_with_id["match_method"].value_counts(dropna=False))
print("\nTotal adresses mappées :", core_with_id["municipal_id"].notna().sum(), "/", len(core_with_id))

# --- Aperçu des correspondances floues ---
display(core_with_id[core_with_id["match_method"].isin(["fuzzy_strict", "fuzzy_loose"])].head(15))


match_method
exact_key       156
fuzzy_strict     73
<NA>             32
fuzzy_loose       2
Name: count, dtype: int64

Total adresses mappées : 231 / 263


,Adresse_norm,municipal_id,match_method,match_score
136,41 avenue saint-just,3094,fuzzy_strict,100
137,999 avenue mceachran,3096,fuzzy_strict,100
138,600 avenue de la gare-de-triage,1717,fuzzy_loose,87
139,7800 boulevard metropolitain est anjou,1062,fuzzy_loose,89
142,7171 rue bombardier anjou,3032,fuzzy_strict,100
143,8750 avenue chaumont anjou,3035,fuzzy_strict,100
146,183 rue des erables,3052,fuzzy_strict,100
147,1925 rue saint-antoine,3207,fuzzy_strict,100
148,2200 rue saint-antoine,3434,fuzzy_strict,100
152,55 avenue dupras,3068,fuzzy_strict,100


## 5) Agrégations annuelles & moyenne 5 ans

In [6]:
ges_pivot = aggregate_by_year(energy_df, years=YEARS, year_col=year_col)

# Prendre seulement les colonnes dispo dans energy_df
meta_cols = [c for c in ["Adresse_norm","Adresse_civique","Annee_construction","Superficie","Superficie_m2"]
             if c in energy_df.columns]
meta_df = (energy_df[meta_cols]
           .drop_duplicates("Adresse_norm", keep="first"))

# Fusion (annuelles + méta) → export final (moyennes 5 ans)
combined = ges_pivot.merge(meta_df, on="Adresse_norm", how="left")
core_final = finalize_export_5y_means(core_with_id, combined, years=YEARS)

print("Adresses avec ID:", core_final["id"].notna().sum(), "/", len(core_final))
core_final.head()



Adresses avec ID: 231 / 263


,id,Adresse,Annee_construction,Superficie,GES,Electricite_GJ_moyenne_5_ans,Gaz_naturel_GJ_moyenne_5_ans,Mazout_GJ_moyenne_5_ans,Conso_totale_GJ,GES_m2
0,<NA>,1 Circt Gilles-Villeneuve,1967,NaN,1.620934,5050.148573,NaN,NaN,5050.148573,NaN
1,95,1000 Avenue Émile-Journault E,1976,49577.0,137.877328,41056.721680,3282.200000,NaN,44338.921680,0.002781
2,8324,10125 Rue Parthenais,1957,NaN,111.640000,271.496360,2194.850000,NaN,2466.346360,NaN
3,978,10300 Rue Lajeunesse,1948,6349.0,87.233943,2098.078400,1588.200000,NaN,3686.278400,0.013740
4,2547,1033 Rue Rachel E,1990,3025.0,27.193916,2643.912880,494.933333,NaN,3138.846213,0.008990


## 6) Export unique CSV


In [7]:
OUT = (Path.cwd().parent / "outputs") if Path.cwd().name == "explorations" else (Path.cwd() / "outputs")
OUT.mkdir(parents=True, exist_ok=True)
out_csv = OUT / "dataset_electricite_nettoye2.csv"
core_final = core_final[core_final["id"].notna()].copy()
core_final.to_csv(out_csv, index=False)
out_csv


WindowsPath('c:/Users/Sarah/Desktop/CODEML/codeml2/outputs/dataset_electricite_nettoye2.csv')

In [8]:
import pandas as pd

old = pd.read_csv(OUT / "dataset_electricite_nettoye.csv")  # ← ton ancien CSV à 218
new = pd.read_csv(OUT / "dataset_electricite_nettoye2.csv")   # ← le nouveau

# 1) Compte
print("OLD:", len(old), "NEW:", len(new))

# 2) Clés de comparaison (id si présent, sinon Adresse)
key = "id" if "id" in old.columns and "id" in new.columns else "Adresse"

# 3) Extra dans NEW (les +14)
extra = new[~new[key].isin(old[key])]
print("Extras:", len(extra))
display(extra[[key, "Adresse"]].head(20))

# 4) Manquants vs NEW
missing = old[~old[key].isin(new[key])]
print("Missing from NEW:", len(missing))
display(missing[[key, "Adresse"]].head(20))

# Après le mapping, regarde la distrib des méthodes
print(core_with_id["match_method"].value_counts(dropna=False))

# Compare les IDs issus du fuzzy/token_set vs exact
mapped_fuzzy = core_with_id[core_with_id["match_method"].isin(["fuzzy_strict","fuzzy_loose","token_set"])]
print("Matches 'non-exact':", len(mapped_fuzzy))
display(mapped_fuzzy.head(10))


OLD: 218 NEW: 231
Extras: 10


,id,Adresse
16,3211,12000 boul Rolland
34,3111,1380 Rue Église
54,3052,183 Rue Érables
55,3052,183 rue des Érables
145,3148,5300 boul Robert
162,1717,"600, ave de la Gare-de-Triage"
164,3457,6025 Boul. Métropolitain E
194,3151,7655 Colbert
195,3151,7655 Rue Colbert
199,3149,7755 rue Colbert


Missing from NEW: 2


,id,Adresse
163,109,1350 Rue Carrières
202,121,5354 Av. Gatineau


match_method
exact_key       156
fuzzy_strict     73
<NA>             32
fuzzy_loose       2
Name: count, dtype: int64
Matches 'non-exact': 75


,Adresse_norm,municipal_id,match_method,match_score
136,41 avenue saint-just,3094,fuzzy_strict,100
137,999 avenue mceachran,3096,fuzzy_strict,100
138,600 avenue de la gare-de-triage,1717,fuzzy_loose,87
139,7800 boulevard metropolitain est anjou,1062,fuzzy_loose,89
142,7171 rue bombardier anjou,3032,fuzzy_strict,100
143,8750 avenue chaumont anjou,3035,fuzzy_strict,100
146,183 rue des erables,3052,fuzzy_strict,100
147,1925 rue saint-antoine,3207,fuzzy_strict,100
148,2200 rue saint-antoine,3434,fuzzy_strict,100
152,55 avenue dupras,3068,fuzzy_strict,100
